# Laboratorio 1: Limpieza de Datos con Python

- Prof. Dr. Rodrigo Salas
- Escuela de Ingeniería Civil Biomédica. 
- Universidad de Valparaíso

El apunte fue realizado siguiente el sitio: https://realpython.com/python-data-cleaning-numpy-pandas/

En este laboratorio trabajaremos con datos "sucios" que procederemos a limpiarlos utilizando las herramientas de numpy y pandas.

In [ ]:
import numpy as np
import pandas as pd

En este laboratorio utilizaremos los siguientes conjuntos de datos:
- **BL-Flickr-Images-Book.csv** Archivo CSV que contiene información de los libros de una biblioteca británica.
- **university_towns.txt** Archivo de texto que contiene el nombre los pueblos universitarios de USA.
- **olympics.csv** Archivo CSV que resume la participación de todos los países en las olimpiadas de verano e invierno

## 1. Eliminando columnas en un DataFrame

La función drop() de pandas permite eliminar las columnas que no son necesarias.

In [ ]:
df = pd.read_csv('BL-Flickr-Images-Book.csv')
df.head()

Procedemos a eliminar algunas columnas

In [ ]:
to_drop = ['Edition Statement',
...            'Corporate Author',
...            'Corporate Contributors',
...            'Former owner',
...            'Engraver',
...            'Contributors',
...            'Issuance type',
...            'Shelfmarks']
df.drop(to_drop, inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
# df.drop(columns=to_drop, inplace=True)

## 2. Cambiando el index del DataFrame

El índice de PANDAS extiende la funcionalidad de los arreglos de Numpy para permitier los recortes y el etiquetamiento. En muchas ocaciones, es muy útil usar como índice un campo con único valor  

Para determinar si el identificador es único:

In [ ]:
df['Identifier'].is_unique

Reemplazar un índice existente con esta columna utilizando set_index

In [ ]:
df = df.set_index('Identifier') 
# df.set_index('Identifier', inplace=True)  con la opción implace=True modifica el objeto
df.head()

Se puede acceder a cada registro utilizando loc[] y el índice

In [ ]:
df.loc[206]

Para acceder utilizando la posición con iloc[]


In [ ]:
df.iloc[0]

## 3. Ordenar los campos en los datos

Ahora procederemos a limpiar algunas columnas y colocarlas en un formato uniforme de manera de poder tener mejor entendimiento de la base de datos y asegurar consistencia.

Procederemos a limpiar Fecha de Publicación y Lugar de publciación.

Todos los tipos de datos son objetos dtype.

In [ ]:
df.get_dtype_counts()

In [ ]:
df.loc[1905:, 'Date of Publication'].head(10)

Debemos realizar las siguientes tareas:
- Remover las fechas extras que están en paréntesis: 1879[1878]
- Convertir el rango de datos a la fecha de partida: 1860-63; 1839, 38-54
- Remover las fechas que no están seguro y reemplazarlo por NaN
- Convertir el string nan por el valor NaN

Expresión regular para extraer la fecha:

In [ ]:
regex = r'^(\d{4})'

In [ ]:
extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)
extr.head()

Luego cambiamos el tipo de dato de objeto a numérico

In [ ]:
df['Date of Publication'] = pd.to_numeric(extr)
df['Date of Publication'].dtype

In [ ]:
df['Date of Publication'].isnull().sum() / len(df)

## 4. Combinando los métodos de str con Numpy para limpiar las columnas.

str es para operaciones con strings.

Ahora limpiaremos lugar de publicación.

In [ ]:
df['Place of Publication'].head(10)

In [ ]:
df.loc[4157862]

In [ ]:
df.loc[4159587]

In [ ]:
pub = df['Place of Publication']
london = pub.str.contains('London')
london[:5]

In [ ]:
oxford = pub.str.contains('Oxford')

In [ ]:
df['Place of Publication'] = np.where(london, 'London',
                                      np.where(oxford, 'Oxford',
                                               pub.str.replace('-', ' ')))

In [ ]:
df['Place of Publication'].head(10)

In [ ]:
df.head()

Sería combeniente cambiar el dtype del lugar de publicación a dato categórico.

## 6. Limpieza de toda la base de datos con la función applymap

In [ ]:
university_towns = []
with open('university_towns.txt') as file:
    for line in file:
        if '[edit]' in line:
            # Remember this `state` until the next is found
            state = line
        else:
            # Otherwise, we have a city; keep `state` as last-seen
            university_towns.append((state, line))

In [ ]:
university_towns[:5]

In [ ]:
towns_df = pd.DataFrame(university_towns,
                        columns=['State', 'RegionName'])

towns_df.head()

In [ ]:
def get_citystate(item):
    if ' (' in item:
        return item[:item.find(' (')]
    elif '[' in item:
        return item[:item.find('[')]
    else:
        return item

In [ ]:
towns_df =  towns_df.applymap(get_citystate)

In [ ]:
towns_df.head()

### Renombrando columnas y saltando filas

In [ ]:
olympics_df = pd.read_csv('olympics.csv')
olympics_df.head()

In [ ]:
olympics_df = pd.read_csv('olympics.csv', header=1)
olympics_df.head()

In [ ]:
new_names =  {'Unnamed: 0': 'Country',
              '? Summer': 'Summer Olympics',
              '01 !': 'Gold',
              '02 !': 'Silver',
              '03 !': 'Bronze',
              '? Winter': 'Winter Olympics',
              '01 !.1': 'Gold.1',
              '02 !.1': 'Silver.1',
              '03 !.1': 'Bronze.1',
              '? Games': '# Games',
              '01 !.2': 'Gold.2',
              '02 !.2': 'Silver.2',
              '03 !.2': 'Bronze.2'}
olympics_df.rename(columns=new_names, inplace=True)